In [178]:

# Atslēdz auto-save iespēju
# %autosave 0



In [179]:
# Instalē visas nepieciešāmās "pip" packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install rdfpandas
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install ipywidgets
!{sys.executable} -m pip install rdflib
!{sys.executable} -m pip install SPARQLWrapper
!{sys.executable} -m pip install rdflib-jsonld

In [180]:
# # # Importētie moduļi

# # "Grafiskie" moduļi
import rdfpandas.graph  # tika importeets Anaconda sisteemaa
import pandas as pd  # tika importeets Anaconda sisteemaa
from ipywidgets import widgets   # pievienots GUI (button, textbox u.tml.) + event handler
from IPython.display import clear_output # dod iespēju izdzēst sūnas "Out" saturu, izmantojot komandu

# # Pamata modulis darbam ar semantiskiem datiem
import rdflib # tika importēts Anaconda sistēmā
from rdflib.plugin import register, Parser
register('json-ld', Parser, 'rdflib_jsonld.parser', 'JsonLDParser')
from rdflib import Graph, URIRef, Literal

# # moduļi strukturētu (ne obligāti RDF) datu iegūšānai 
from urllib.request import urlopen, Request
import json

# # modulis SPARQL valodas pielietošanai saistīto datu iegūšanai 
from SPARQLWrapper import SPARQLWrapper, JSON 

# # Papildus moduļi datu apstrādei / analīzei
import datetime # var pārbaudīt, vai objekts ir datums

In [181]:
# # # Sistēmas funkcijas un galvenie mainīgie (galvenokārt, vārdnīcas vai saraksti)

def rand_node(count):
    import random
    import string
    node = ""
    char_set = string.ascii_letters + "0123456789"
    for i in range(count):
        node += random.choice(char_set)
    return node


def val_type(v):
    return type(v).__name__

In [182]:
# # # SPARQL izsaukumi

sparql_lang_wrapper = "https://databus.dbpedia.org/repo/sparql"
sparql_lang_query = """
    PREFIX dataid: <http://dataid.dbpedia.org/ns/core#>
    PREFIX dct:    <http://purl.org/dc/terms/>
    PREFIX dcat:   <http://www.w3.org/ns/dcat#>
    PREFIX db:     <https://databus.dbpedia.org/>
    PREFIX rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#>

    SELECT DISTINCT ?l WHERE {
      ?s a dataid:Dataset ; dataid:artifact <https://databus.dbpedia.org/dbpedia/generic/labels> ; dcat:distribution ?d. 
      ?d <http://dataid.dbpedia.org/ns/cv#lang> ?l.
    } 
    LIMIT 1000
"""
def sparql_result(wrapper, query):    
    arr = []

    sparql = SPARQLWrapper(wrapper)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
#     results
    for result in results["results"]["bindings"]:
        arr.append(result["l"]["value"])
    return arr

# def sparql_result(wrapper, query):    

sparql = SPARQLWrapper(sparql_lang_wrapper)
sparql.setQuery(sparql_lang_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
# results



# print(sparql_result(sparql_lang_wrapper, sparql_lang_query))

In [183]:
# represents_type(s) - funkcija, kas pārbauda "Literal" vērtību, pārbauda, 
# kāds varētu būt tā datutips un izvada tā nosaukumu, balstoties uz 
# XMLSchema datutipiem (pagaidām tikai trīs tipi, 
# kas tika atrasti JSON struktūrā)



lang_arr = sparql_result(sparql_lang_wrapper, sparql_lang_query)
def represents_type(s):
    # OWL(varbūt) vārdnīcas
    
    uri_prefix_arr = ["http://","https://","uri:","_:"]
    try:
        datetime.datetime.strptime(s[0:10], "%Y-%m-%d")
        return "date"
    except:
        try:
            if(val_type(s) == "list"):
                return "list"
            else:
                raise ValueError() 
        except:   
            try:
                if(val_type(s) == "str"):
                    if(("." in s) and (float(s))):
                        return "decimal"
                    else:
                        raise ValueError()
                else:
                    if (isinstance(s, float)):
                        return "decimal"
                    else:
                        raise ValueError()      
            except:
                try:
                    if(isinstance(s, int) and str(s) != "True" and str(s) != "False"):
                        return "nonNegativeInteger"
                    else:
                        raise ValueError()
#                     int(s)
#                     return "nonNegativeInteger"
                except:
                    try:
                        for pref in uri_prefix_arr:
                            if(s.startswith(pref)):
                                return "uri"
                        else:
                            raise ValueError()
                    except:
                        try:
                            if(s in lang_arr):
                                return "lang"
                            else:
                                raise ValueError()
                        except: 
                            try:
                                for lang in lang_arr:
                                    if(s.startswith(lang+"_")):
                                        return "lang_ext"
                                else:
                                    raise ValueError()
                            except: 
                                try:
                                    if(str(s) == "True" or str(s) == "False"):
                                        return "boolean"
                                    else:
                                        raise ValueError()
                                except: 
                                    return "langString"




specific_ns = {"dbr" : rdflib.term.URIRef("http://dbpedia.org/resource/")}

# graph_to_prefix_table__DataFrame(rdf_graph) - funkcija, kas 
# saīsina URI, izmantojot PREFIX vērtības un pārveido datus, 
# lai tie tiktu smukāk attēloti "DataFrame" tabulā
def graph_to_prefix_table__DataFrame(rdf_graph):
    subject_array = []
    predicate_array = []
    object_array = []
    dict = {}

    # Pievieno prefiksus no:
    # DBPedia turtle formāta faila;
    for ns in rdf_graph.namespaces():
        dict[ns[0]] = ns[1]
    # speciālo prefiksu vārdnīcas
    for key,val in specific_ns.items():
        dict[key] = val

    # saīsina URI, izmantojot PREFIX vērtības
    for subj, pred, obj in rdf_graph:
#         for key,val in dict.items(): 
#             if (subj.startswith(val)):
#                 subj = subj.replace(val, key + ":")           
#             if (pred.startswith(val)):
#                 pred = pred.replace(val, key + ":")   
#             if (obj.startswith(val)):
#                 obj = subj.replace(val, key + ":")  

        subject_array.append( subj )
        predicate_array.append( pred )
        object_array.append( obj )

    # rindu (ierakstu) skaits tabulā      `
    pd.set_option('display.max_rows', len(subject_array)) 
    # kolonnas max iespējamais ievadāmo simb skaits
    pd.options.display.max_colwidth = max_width  
    
    return {'Subject': subject_array, 'Predicate': predicate_array, 'Object': object_array}


# represents_type(True)

In [184]:
# # # Kitsu API izveidotās funkcijas:
lang_type = ["lang", "lang_ext"]
def schema_val(v, k=""):
    
#     if(v==None or v==[] or v=={}):
# #     if(v==None):
#         v = "Null"
    
    
    if(represents_type(v) == "list"):
        v_modif = {}
        for v_el in v:
            schema_type = "https://www.w3.org/2001/XMLSchema#"
            schema_type += represents_type(v_el)
            v_modif.update({"@value": v_el, "@type": schema_type})
    else: 
        if(represents_type(k)=="lang"):
            v_modif = {"@value": v, "@language": k}
        elif(represents_type(k)=="lang_ext"):
            v_modif = {"@value": v, "language": k}     
        else:
            if(represents_type(v) != "str"):
                if(represents_type(v) == "uri"):
                    v_modif = {"@id": v}
                else:
                    schema_type = "https://www.w3.org/2001/XMLSchema#"
                    schema_type += represents_type(v)
                    v_modif = {"@value": v, "@type": schema_type}
            else: 
                schema_type = "http://www.w3.org/2001/XMLSchema#"
                schema_type += represents_type(v)
                v_modif = {"@value": v, "@type": schema_type}
    return v_modif

uri_domain = "uri:kitsu/"
def dbp_uri(key, rel_type = ""):
    if(rel_type != ""):
        rel_type += "/"
    return uri_domain + rel_type +  key

In [185]:
headers = {"User-Agent": 
           """
           Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) 
           Chrome/41.0.2228.0 Safari/537.3"
           """
           }
           
def get_kitsu_data_by_uri(req_url, headers):
    try:
        req = Request(url=req_url, headers=headers) 
        json_api_content = urlopen(req).read().decode("UTF-8")
        
        kitsu_json_data = json.loads(json_api_content)["data"]
        
        if(kitsu_json_data == None):
            data = []
        else:
            if(val_type(kitsu_json_data) == "dict"):
                data = [kitsu_json_data]
            else:
                data = kitsu_json_data
        return data
    except:
        return []



def url_with_filter_and_sort(uri, page_limit_val, page_offset_val):
#     sort_v = sort_val
    page_limit_v = page_limit_val
    page_offset_v = page_offset_val
    
#     sort = "sort=" + sort_v
    page_limit = "page[limit]=" + page_limit_v
    page_offset = "page[offset]=" + page_offset_v
    
#     uri += "?" +  sort + "&" + page_limit + "&" + page_offset
    uri += "?" + page_limit + "&" + page_offset
    return uri

def get_max_data(uri):
#     sort_v = "number" 
    page_limit_v = 20
    page_offset_v = 0
    

    data = []
    uri = url_with_filter_and_sort(uri, str(page_limit_v), str(page_offset_v))
    tmp_data = get_kitsu_data_by_uri(uri, headers)
    if(len(tmp_data)==page_limit_v): 
        while(tmp_data != []):
            data += tmp_data
            page_offset_v += page_limit_v
#             uri = url_with_filter_and_sort(uri, str(page_limit_v), str(page_offset_v))
            uri = url_with_filter_and_sort(uri, str(page_limit_v), str(page_offset_v))
            tmp_data = get_kitsu_data_by_uri(uri, headers)
    else:
        data += get_kitsu_data_by_uri(uri, headers)
    return data

In [186]:
def not_empty(val):
    if(val != "" and val != None and val != [] and val_type(val) != "NoneType" 
    and val != {}):
        return True
    else:
        return False

domain_edge = ""



 
def spec_val(key, val):
    if("youtube" in key.lower() or "yt" in key.lower()):
        return "https://www.youtube.com/watch?v=" + val
    else:
        return val

def check_empty_val(data):
    tmp_chained = {}
    for key, val in data.items():
        if("@value" in val):
            if(not_empty(val["@value"])):
                tmp_chained.update({key:val})
    return tmp_chained



def get_chained_dict_data(val, key_uri):
    tmp_dict = {}
    for chn_key, chn_val in val.items():
        chn_key_tmp = key_uri + "/" + chn_key
        if(val_type(chn_val) != "dict"):
            if(not_empty(chn_val)):
                tmp_dict.update({dbp_uri(chn_key_tmp) : [schema_val(chn_val)]})
        else:
            tmp_chain_dict = get_chained_dict_data(chn_val, chn_key_tmp)
            tmp_dict.update(tmp_chain_dict)
    return tmp_dict
# test_dict = get_chained_dict_data(v,k)
# test_dict = get_chained_dict_data(v_meta,k_meta)
# print(json.dumps(test_dict))
    

    

def new_key_func(long_key, compare_with):
    if(compare_with in long_key):
        if(compare_with + "/" in long_key):
             new_long_key = long_key.replace(compare_with + "/", "")
        elif("/" + compare_with in long_key):
             new_long_key = long_key.replace("/" + compare_with, "")
        else:
             new_long_key = long_key.replace(compare_with + "/", "") 
    else:
        new_long_key =  long_key
    return new_long_key
# print(new_key("uri:kitsu/meta/dimensions/tiny/height", "tiny"))
    

def get_image_data(val):
    result = {}
    tmp_id_storage = []
    tmp_id_data_storage = []
    img_size = ["tiny", "small", "medium", "large", "original"]
    
    if("meta" in val):
        chained_data = get_chained_dict_data(val["meta"], "meta")
        
    for img_key, img_val in val.items():
        if(img_key in img_size):
            
            img_val_uri = img_val + "/" + img_key
            id_dict = schema_val(img_val_uri)
            tmp_id_storage.append(id_dict)
            
            id_data_dict = schema_val(img_val_uri)
            id_data_dict.update({dbp_uri("img_path"):img_val})


            tmp_chained_data = []
            for chn_key, chn_val in chained_data.items():  
                if(img_key in chn_key):
                    new_key = new_key_func(chn_key, img_key)
                    id_data_dict.update({new_key:chn_val})
        tmp_id_data_storage.append(id_data_dict)
    result = {"id_list":tmp_id_storage, "id_data_list":tmp_id_data_storage}
    return result

In [ ]:
def get_certain_article_data(uri, main_data_types, including_rel, rel_second_level):
    data = get_max_data(uri)
    
    dont_need = ["createdAt", "updatedAt"]
    tmp_list_arr = []
    
    if(data != []):

        if("type" in data[0]):
            tmp_rel_type = data[0]["type"]

        for data_el in data:

            tmp_list = {}  

            for key, val in data_el.items():
                
                if(key == "links"):
                    tmp_list.update({"@id":val["self"]})
                elif(key == "type"):
                    tmp_list.update({"@type":dbp_uri(val)}) 
                elif(key == "attributes"): 
                    
                    for attr_key, attr_val in val.items():
                        attr_val = spec_val(attr_key, attr_val)
                        if(val_type(attr_val)=="dict" and attr_val != {}):
                            if(represents_type(next(iter(attr_val))) in lang_type):
                                final_key = dbp_uri(attr_key)
                                tmp_lang_arr = []
                                for lang_key, lang_val in attr_val.items():
                                    if(not_empty(lang_key) and not_empty(lang_val)):
                                        tmp_lang_arr.insert(0,schema_val(lang_val,lang_key)) 
                                tmp_list.update({final_key:tmp_lang_arr})
                            else: 
                                if("image" in attr_key.lower()):
                                    img_data = get_image_data(attr_val)
                                    tmp_list.update({attr_key:img_data["id_list"]})
                                    for img_el in img_data["id_data_list"]:
                                        tmp_list_arr.append(img_el)
                                else:
                                    tmp_chained_data = get_chained_dict_data(
                                        attr_val, attr_key)

                                    tmp_chained_data = check_empty_val(tmp_chained_data)
                                    tmp_list.update(tmp_chained_data)
                        else:
                            if(attr_key not in dont_need):
                                if(not_empty(attr_val)):
                                    final_key = dbp_uri(attr_key, tmp_rel_type)
                                    tmp_list.update({final_key:[schema_val(attr_val)]})   

                elif(key == "relationships"): 
                    if(tmp_rel_type in main_data_types):
                        for rel_key, rel_val in val.items():
                            if(rel_key in including_rel):    
                                snd_level_rel_uri = rel_val["links"]["related"]

                                final_key = dbp_uri(rel_key, key)

                                tmp_list_arr_2nd_level = get_certain_article_data(
                                    snd_level_rel_uri, main_data_types, 
                                    including_rel, rel_second_level)
                                tmp_rel_val_arr = []
                                for el_1st in tmp_list_arr_2nd_level:
                                    if("@type" in el_1st):
                                        if(rel_key in el_1st["@type"]):
                                            el_1st_id = schema_val(el_1st["@id"])
                                            tmp_rel_val_arr.append(el_1st_id)
                                    
                                    tmp_list_arr.append(el_1st)
                                tmp_list.update({final_key:tmp_rel_val_arr})


                    elif(tmp_rel_type not in main_data_types 
                    and tmp_rel_type in rel_second_level):

                        split_arr = rel_second_level[tmp_rel_type].split("/")
                        for rel_key, rel_val in val.items():

                            if(tmp_rel_type in rel_second_level 
                            and rel_key in rel_second_level[tmp_rel_type]):
                                snd_level_rel_uri = rel_val["links"]["related"]

                                final_key = dbp_uri(rel_key, key)
                                                                
                                tmp_list_arr_2nd_level = get_certain_article_data(
                                    snd_level_rel_uri, main_data_types, 
                                    including_rel, rel_second_level)
                                tmp_rel_val_arr = []

                                for el_1st in tmp_list_arr_2nd_level: 
                                    if("@type" in el_1st):
                                        for arr_el in split_arr:
                                            if(arr_el in el_1st["@type"]):
                                                el_1st_id = schema_val(el_1st["@id"])
                                                tmp_rel_val_arr.append(el_1st_id)
                                    
                                    tmp_list_arr.insert(0,el_1st)
                                tmp_list.update({final_key:tmp_rel_val_arr})



            tmp_list_arr.insert(0,tmp_list)
    return tmp_list_arr


def get_article_data(uri):
    
    main_data_types = ["anime", "manga", "drama"]
    
    including_rel = ["genres", "categories", "castings", "reviews"]
    rel_second_level = {"castings":"people/person", "reviews": "user/users"}
    
    if("anime" in uri):
        including_rel += ["episodes", "animeProductions", 
                          "animeCharacters", "animeStaff"]

        rel_second_level.update({"animeCharacters": "character",
                                 "animeStaff": "people", 
                                 "animeProductions":"producer"})
    
    elif("manga" in uri):
        
        including_rel += ["chapters", "characters", "staff"]
        rel_second_level.update({"characters": "mediaCharacters/character",
                                 "staff": "people"})

    return get_certain_article_data(uri, main_data_types, including_rel, rel_second_level)

# Piemēri:
######################## Anime

# ex_uri = "https://kitsu.io/api/edge/anime/10/genres"
# ex_uri = "https://kitsu.io/api/edge/anime/10/castings"
# ex_uri = "https://kitsu.io/api/edge/people/485"
# ex_uri = "https://kitsu.io/api/edge/anime/10/reviews"
# ex_uri = "https://kitsu.io/api/edge/anime/10/anime-characters"
# ex_uri = "https://kitsu.io/api/edge/anime/10/episodes"
# ex_uri = "https://kitsu.io/api/edge/episodes/450"


ex_uri = "https://kitsu.io/api/edge/anime/10" 

######################## Manga

# ex_uri = "https://kitsu.io/api/edge/manga/35/genres"
# ex_uri = "https://kitsu.io/api/edge/manga/35/castings"
# ex_uri = "https://kitsu.io/api/edge/manga/35/characters"
# ex_uri = "https://kitsu.io/api/edge/manga/35/chapters"

# ex_uri = "https://kitsu.io/api/edge/manga/35"

################################################

# ex_uri = "https://kitsu.io/api/edge/anime/3"

kitsu_data_ex = get_article_data(ex_uri)
# print(json.dumps(kitsu_data_ex))
# # 17022

In [165]:
# # Kitsu API:

g_kitsu = rdflib.Graph()
g_kitsu_data = g_kitsu.parse(data=bytes(json.dumps(kitsu_data_ex),encoding='utf8'), format='json-ld')

# # DBPedia API:

g_DBpedia = rdflib.Graph()
g_DBpedia.load("http://dbpedia.org/resource/Death_Note")
g_DBpedia.load("http://dbpedia.org/resource/KonoSuba")

In [166]:
# # # Pārveido izvada vizuālo izskatu 


# Palielina ekrāna izmēru kodam un izvadam līdz ekrāna izmēŗam - 100%
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# balstoties uz subj/pred/obj simbolu garumiem, tiks noteikts 
# maksimālais simbolu skaits visām tabulas kolonnas šūnām
max_width = 10
for subj, pred, obj in g_DBpedia: 
    if(len(subj) > max_width):
        max_width = len(subj)
    elif(len(pred) > max_width):
        max_width = len(pred)
    elif(len(obj) > max_width):
        max_width = len(obj) 

In [177]:
### informācijas savākšans no DBPedia linka

d1 = graph_to_prefix_table__DataFrame(g_DBpedia)
df1 = pd.DataFrame(data=d1)
df1 = df1.style.set_properties(**{'text-align': 'left'})

d2 = graph_to_prefix_table__DataFrame(g_kitsu_data)
df2 = pd.DataFrame(data=d2)
df2 = df2.style.set_properties(**{'text-align': 'left'})

In [176]:
# Parādās pogas "DBPedia" un "Kitsu API", kuras piespiežot parādās dati par mēdiju materiāliem iegūtus no konkrētā API

dbp_button = widgets.Button(description="DBPedia")
display(dbp_button)
kitsu_button = widgets.Button(description="Kitsu API")
display(kitsu_button)

out = widgets.Output()
def on_button_clicked(b):
    with out:
        clear_output(True)
        if(b.description == "DBPedia"):
            clear_output(True)
            display(df1)
        else:
            clear_output(True)
            display(df2)
#             display(df3)

dbp_button.on_click(on_button_clicked)
kitsu_button.on_click(on_button_clicked)

out

Button(description='DBPedia', style=ButtonStyle())

Button(description='Kitsu API', style=ButtonStyle())

Output()